In [1]:
import os
import glob
import cv2
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from concurrent.futures import ThreadPoolExecutor

c:\Users\PhucHai\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
root_path = 'D:\\CS231\\Final_Project'
root_list = os.listdir(root_path)
train_path = os.path.join(root_path, 'train')
test_path = os.path.join(root_path, 'test')

In [3]:
image_train = []
y_train = []

def compute_hog_feature(image, orientations, pixels_per_cell, cells_per_block):
    if image is not None:
        hog_feature, _ = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
        return hog_feature
    return None

for index, image_dir in enumerate(os.listdir(train_path)):
    label = index
    image_dir_path = os.path.join(train_path, image_dir)
    image_paths = glob.glob(os.path.join(image_dir_path, "*.jpg"))
    
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            img_GRAY = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            img_GRAY = cv2.resize(img_GRAY, (128, 128))
            image_train.append(img_GRAY)
            y_train.append(label)

orientations_list = [2, 6, 9, 12]
pixels_per_cell_list = [(4, 4), (8, 8), (16, 16)]
cells_per_block_list = [(2, 2), (3, 3), (4, 4)]

best_params = None
best_accuracy = 0

def evaluate_hog_parameters(orientations, pixels_per_cell, cells_per_block):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(compute_hog_feature, img, orientations, pixels_per_cell, cells_per_block) for img in image_train]
        X_train_hog = [future.result() for future in futures if future.result() is not None]

    knn_classifier = KNeighborsClassifier()

    param_grid = {'n_neighbors': range(1, 31)}

    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)\
    
    grid_search = GridSearchCV(knn_classifier, param_grid, cv=kf)
    
    grid_search.fit(X_train_hog, y_train)

    results = grid_search.cv_results_

    best_mean_score = -1
    best_parameters = None
    for mean_score, params in zip(results['mean_test_score'], results['params']):
        if best_mean_score < mean_score:
            best_mean_score = mean_score
            best_parameters = params

    return best_mean_score, best_parameters, orientations, pixels_per_cell, cells_per_block

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(evaluate_hog_parameters, orientations, pixels_per_cell, cells_per_block)
               for orientations in orientations_list
               for pixels_per_cell in pixels_per_cell_list
               for cells_per_block in cells_per_block_list]
    
    for future in futures:
        mean_score, params, orientations, pixels_per_cell, cells_per_block = future.result()
        print(f"Testing orientations={orientations}, pixels_per_cell={pixels_per_cell}, cells_per_block={cells_per_block}")
        print(f"Best accuracy: {mean_score}")
        if mean_score > best_accuracy:
            best_accuracy = mean_score
            best_params = {
                'orientations': orientations,
                'pixels_per_cell': pixels_per_cell,
                'cells_per_block': cells_per_block,
                'n_neighbors': params['n_neighbors']
            }

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")

Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(2, 2)
Best accuracy: 0.659233963098543
Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(3, 3)
Best accuracy: 0.6988465223693584
Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(4, 4)
Best accuracy: 0.7166879591184925
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(2, 2)
Best accuracy: 0.7345146549394392
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(3, 3)
Best accuracy: 0.8003783504901358
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(4, 4)
Best accuracy: 0.8291108763481807
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(2, 2)
Best accuracy: 0.7592683585976464
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(3, 3)
Best accuracy: 0.8301083458221754
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(4, 4)
Best accuracy: 0.8469449426332212
Testing orientations=6, pixels_per_cell=(4, 4), ce